# Task 2c

In [39]:
import torch
import os
import networkx as nx
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from scipy.stats import describe
import ncut

### Loading the graphs

Graph loading functions, could be changed to load multigraph directly from file

In [40]:
def csv_to_graph(path, threshold=0.3):
    # load in csv as dataframe
    df = pd.read_csv(path,header=None)
    
    # threshold dataframe and remove diagonal
    A = (df>threshold).astype(int) - pd.DataFrame(np.identity(df.shape[0]))
         
    # convert to graph
    G = nx.from_pandas_adjacency(A)
    
    return G

def load_graphs(folder):
    #load in all graphs in folder
    G1s, G2s = [],[]

    for i in range(1,61):
        filename = f'{folder}/p{i:03}_1.csv'
        G1s.append(csv_to_graph(filename))

        filename = f'{folder}/p{i:03}_2.csv'
        G2s.append(csv_to_graph(filename))
        
    return G1s, G2s

G1s, G2s = load_graphs("FC")

In [41]:
def graph_list_to_multigraph(graphs):
    G = nx.MultiGraph()
    for subgraph in graphs:
        G.add_nodes_from(subgraph.nodes)
        G.add_edges_from(subgraph.edges)
    return G

In [42]:
G1 = graph_list_to_multigraph(G1s)
G2 = graph_list_to_multigraph(G2s)

### Loading the embeddings

In [43]:
EMBEDDING_DIR = "embedding_pickles"
G1_PATH = "best_G1_DMGI.pkl"
G2_PATH = "best_G2_DMGI.pkl"

In [44]:
g1_model = torch.load(os.path.join(EMBEDDING_DIR, G1_PATH))
g1_embedding = g1_model['H'].squeeze()

g2_model = torch.load(os.path.join(EMBEDDING_DIR, G2_PATH))
g2_embedding = g2_model['H'].squeeze()

### Loading additional clinical data

In [45]:
clinical_csv = pd.read_csv("clinical.csv")
clinical_csv

,ID,SEX,MS_TYPE,AGE,EDSS,DATASET,DIAG_YEARS,BMI,THERAPY,outliers_V1,outliers_V2
0,p001,M,primary_progressive,40,7,2,11,19.918,Functional_electric_stimulation,0,14
1,p002,M,secondary_progressive,44,7,2,17,20.529,Motor_Program_Activating_Therapy,4,0
2,p003,M,primary_progressive,51,6,2,21,25.249,Functional_electric_stimulation,4,0
3,p004,M,relapsing_remitting,29,3,2,9,21.500,Functional_electric_stimulation,4,2
4,p005,F,secondary_progressive,60,6,2,21,21.411,Functional_electric_stimulation,0,4
5,p006,M,secondary_progressive,68,4,2,7,21.605,Motor_Program_Activating_Therapy,2,3
6,p007,F,relapsing_remitting,36,4,2,9,19.370,Functional_electric_stimulation,6,7
7,p008,F,relapsing_remitting,32,4,2,4,19.141,Functional_electric_stimulation,5,7
8,p009,F,relapsing_remitting,35,4,1,20,16.436,Motor_Program_Activating_Therapy,3,0
9,p010,M,secondary_progressive,54,6,2,12,23.148,Motor_Program_Activating_Therapy,20,6


## Validation of our NCut implementation
Validates our implementation of NCut against networkx by comparing the NCut values of random binary graph partitions.

In [48]:
# binary partitions validation on multigraphs

identical = True
graphs = [G1, G2]

n_checks = 10
precision = 1e-10

for i in range(n_checks):
    
        np.random.seed(150896)

        print(f"Checking random partition no. {i}")
        clustering = np.random.randint(0, 2, size=len(G1.nodes))
        
        nodes1 = np.where(clustering==0)[0]
        nodes2 = np.where(clustering==1)[0]

        for graph in graphs:
            nx_result = nx.normalized_cut_size(graph, nodes1, nodes2)
            our_result = ncut.ncut_multigraph(graph, nodes1, nodes2)
            # our_k_result = ncut.k_ncut_multigraph(graph,  np.asarray([np.where(clustering==0)[0], np.where(clustering==1)[0]]))
            our_k_result = ncut.k_ncut_multigraph(graph, [nodes1, nodes2])
            
            if abs(our_result - nx_result) > precision or abs(our_k_result - nx_result) > precision:
                identical = False
                break
        
        else:
            continue
        
        break

print(f"\nOur results were{' ' if identical else ' not '}identical to those of networkx for {n_checks} random partitions.")

Checking random partition no. 0


In [ ]:
# binary partitions validation on single graphs

identical = True
graphs = [G1s, G2s]

n_checks = 10
precision = 1e-10

for i in range(n_checks):
    
        print(f"Checking random partition no. {i}")
        clustering = np.random.randint(0, 2, size=len(G1.nodes))
        
        for graph in graphs:
            for subgraph in graph:
                nx_result = nx.normalized_cut_size(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                our_single_result = ncut.ncut_single_graph(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                # our_result = ncut.ncut_multigraph(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                our_k_result = ncut.k_ncut_multigraph(subgraph,  [np.where(clustering==0)[0], np.where(clustering==1)[0]])

                if abs(our_result - nx_result) > precision or \
                    abs(our_k_result - nx_result) > precision or \
                    abs(our_result - nx_result) > precision:
                        
                    identical = False
                    break
        
        else:
            continue
        
        break

print(f"\nOur results were{' ' if identical else ' not '}identical to those of networkx for {n_checks} random partitions.")

In [ ]:
# TODO: can k-way n-cut be validated?

## Clustering the data

In [ ]:
def ncut_by_k_partition(graph, partition):
    """Helper function to generate the right input structure for ncut.k_ncut_multigraph"""
    node_lists = np.asarray([np.where(partition == value)[0] for value in np.unique(partition)], dtype=object)
    return ncut.k_ncut_multigraph(graph, node_lists)

In [ ]:
n_clusters = range(2, 20)

kmeans_ncut_results = []

for n in n_clusters:
    kmeans = KMeans(n_clusters=n)
    clustering1 = kmeans.fit_predict(g1_embedding)
    clustering2 = kmeans.fit_predict(g2_embedding)
    g1_ncut = ncut_by_k_partition(G1, clustering1)
    g2_ncut = ncut_by_k_partition(G2, clustering2)
    kmeans_ncut_results.append((f"KMeans n_clusters={n}", g1_ncut, g2_ncut, clustering1, clustering2))

In [ ]:
for result in kmeans_ncut_results:
    print(result[:3])

In [ ]:
# trying to normalize the k-way n-cut values but unfortunately they are still dependent on k
for k, result in zip(range(2,20), kmeans_ncut_results):
    print(result[0], result[1]/k, result[2]/k)

## Comparison of partitions

### Comparison of NCut-values
The NCut-values of the clusterings are slightly higher for the first graph.

In [ ]:
kmeans_ncut_results = [list(res) for res in kmeans_ncut_results]
result_diffs = np.asarray([res[1] - res[2] for res in kmeans_ncut_results])
describe(result_diffs)

### NMI scores
Compares how similar clusterings on G1 and G2 are by calculating the NMIs between them.

In [ ]:
descriptions = [res[0] for res in kmeans_ncut_results]

kmeans_nmis = [normalized_mutual_info_score(res[3], res[4]) for res in kmeans_ncut_results]
for res, nmi in zip(descriptions, kmeans_nmis):
    print(f"{res} NMI: {nmi}")

### Comparison by attributes in clinical.csv
In this section we investigate whether the attributes in clinical.csv (such as the age) have an influence on the NCut values of the partitions. To do that, we interpret the column values of clinical.csv as cluster labels, and compute the NCut valus of the graph layers (i.e. the patients) individually. Afterwards, we group the NCut values by cluster labels and compare descriptive statistics between the groups.

In [ ]:
comparison_attributes = [
    ['sex', clinical_csv['SEX']],
    ['age over 50', clinical_csv['AGE'] > 50],
    ['BMI over 25', clinical_csv['BMI'] > 25],
    ['MS type', clinical_csv['MS_TYPE']],
    ['therapy', clinical_csv['THERAPY']]
]

In [ ]:
# takes a while to run

individual_ncuts = []

for res in kmeans_ncut_results:
    
    G1_ncuts = []
    G1_partition = res[3]
    for patient in G1s:
        G1_ncuts.append(ncut_by_k_partition(patient, G1_partition))

    G2_ncuts = []
    G2_partition = res[4]
    for patient in G2s:
        G2_ncuts.append(ncut_by_k_partition(patient, G2_partition))
        
    individual_ncuts.append([res[0], G1_ncuts, G2_ncuts])

In [ ]:
attribute_statistics = []

for part_description, G1_ncuts, G2_ncuts in individual_ncuts:
    for comp_description, labels in comparison_attributes:
        
        label_ncuts_g1 = {}
        label_ncuts_g2 = {}
        
        for label in np.unique(labels):
            indices = np.where(labels == label)[0]
            
            # note: if desired, more descriptive statistics could be saved here
            stats = describe(np.asarray(G1_ncuts)[indices])
            label_ncuts_g1[label] = {'mean': stats.mean, 'variance': stats.variance}
            
            stats = describe(np.asarray(G2_ncuts)[indices])
            label_ncuts_g2[label] = {'mean': stats.mean, 'variance': stats.variance}
            
        attribute_statistics.append({'partition_name' : part_description, 'attribute_name' : comp_description,
                                    'G1' : label_ncuts_g1, 'G2' : label_ncuts_g2})

In [ ]:
for entry in attribute_statistics:
    
    print("Partition:", entry['partition_name'])
    print("Attribute:", entry['attribute_name'], "\n")
    
    for key in entry['G1'].keys():
        print(key)
        print("G1 mean ncut:", entry['G1'][key]['mean'])
        print("G2 mean ncut:", entry['G2'][key]['mean'])
        print()
        
    print("-------------------")

Comparing the mean NCuts for n_clusters=2 between patients of different ages, it seems that patients under 51 years have a lower average ncut value than those over 50 in graph 1, but a higher average ncut value in graph 2. A higher ncut value means that the partitions are more similar to each other. Based on this result, we can speculate that ???\
**insert more analysis here**

# Task 2d